# Quickstart

To get started with LSDB we will demonstrate a very common workflow. It consists of crossmatching a small set of objects of your interest with a large survey catalog stored in HiPSCat format (e.g. Gaia), and saving the final result.

The first thing you need to do is to import our package.

In [ ]:
import lsdb

Create a Pandas Dataframe containing the objects you want to crossmatch with Gaia.

In [ ]:
import pandas as pd

# The coordinates (ra, dec) for our objects of interest
objects = [(180.080612, 9.507394), (179.884664, 10.479632), (179.790319, 9.551745)]

objects_df = pd.DataFrame(objects, columns=["ra", "dec"])
objects_df

Now that the data is in a DataFrame you can create an LSDB catalog to have it in HiPSCat format.

In [ ]:
my_object_catalog = lsdb.from_dataframe(objects_df, catalog_name="my_object_catalog", catalog_type="object")
my_object_catalog

Next, read the catalog we want to crossmatch with. Because we will download it from the web we decided to go with a small half degree region of Gaia. In this case we need to install __aiohttp__ but if your catalog happens to be present in local storage you can call `read_hipscat` directly.

In [ ]:
!pip install aiohttp --quiet

In [ ]:
small_gaia = lsdb.read_hipscat("https://epyc.astro.washington.edu/~lincc-frameworks/half_degree_surveys/gaia")
small_gaia

Let's crossmatch them! As a result we will have a catalog with the objects in the small Gaia that match our initial objects of interest, according to a specified maximum distance. We will use the default K nearest neighbors algorithm with `k=1` and a maximum separation distance of 1 arcsecond.

In [ ]:
result = my_object_catalog.crossmatch(
    small_gaia, n_neighbors=1, radius_arcsec=1 * 3600, require_right_margin=False
)

As expected, we obtained at most 1 match for each of our initial objects:

In [ ]:
result.compute()

Let's say you're done with your workflow. Generally, it is a good idea to save your catalog to disk to prevent regenerating it later on. Our catalog interface exposes the `to_hipscat` API for that, you just need to provide it with a path for the `target directory` and a `catalog name`.

In [ ]:
result.to_hipscat("lsdb_catalogs", catalog_name="my_object_catalog_x_small_gaia")